# **Exploratory Data Analysis**: FlipTop

## Data Wrangling

In [4]:
import yt_dlp as yt
import pandas as pd
import numpy as np
import requests as req
from bs4 import BeautifulSoup as Soup

print("Environment working, yeah?")

Environment working, yeah?


Small beginnings.